In [9]:
prompt_facts_final= """[Role]: You are an expert semantic parser and logical reasoning system. 
[Your task] convert the natural language spatial scene into an Answer Set Programming (ASP) program by extracting atomic facts from the context and formulating queries from the questions.

#step 1. Atomic Facts Extraction:
Carefully analyze the provided context to identify all blocks, objects, and their spatial relationships in EACH Sentence, and then represent using the one of three predicates with exact number of arguments is/3, block/2, block/1:
  a) block(name). eg. block(a;b;c).
  b) object(unique_identifier, block_located). unique_identifer = size_color_shape_(number if necessary), omit any attribute if the attribute is missing. eg. object(large_blue_triangle, a). object(blue_oval, a).  object(large_blue_triangle_3, c).
  c) is(a, relation, b), eg. is(large_blue_triangle, left, blue_oval), is(a, left, b). 
  relation must be one of 10 relations: left, right, above, below, near, far, front, behind, adjacent, disconnected,touching. 
  Incorrect: is(medium_blue_square, far_left, yellow_square) Correct: is(medium_blue_square, far, yellow_square). is(medium_blue_square, left, yellow_square).
  d)  If the context does not explicitly state a relationship between two objects in the block, do infer their relationship from the touching edge of block or adjacent information.
   eg.“The big blue circle is touching the bottom edge of A, and the green circle is touching the top edge of A.” should be represented as is(green_circle, above, big_blue_circle).
## Donot miss any fact information from the  context, especially  the unclear spatial relations between objects within one object.

## step 2. Query Formulation for the question

Query format: query(Argument) :- conditions using is/3, object/2, block/1.
For Yes/No questions:  query(yes) :- <conditions_for_yes>. query(no) :- not query(yes).
For Wh-Questions:  query(Variable) :- <conditions>, Variable = <result>

a) When a query involves an object that's not fully specified, list all possible objects that match the given attributes in the facts. Use the semicolon (;) to separate multiple possibilities. 
eg. query(Object) :- object(Object, Block), Object = red_small_circle; Object = red_square; Object = red_large_triangle.

b) Important: Ensure the consistent naming of variables in object/2, is/3 across query head and query body, between the previous generated facts and queries.

Example: 
Context: There are three blocks called A, B, and C. A is on top of B and B is on top of C. In A there are 3 circles: a small red one, a green one, and a big blue one. The small red and green circles are above the big blue circle. The big blue circle is touching the bottom edge of A, and the green circle is touching the top edge of A. In B there is a orange triangle, Which is above a purple rectangle. The purple rectangle is touching the bottom edge of B. In C, there are 3 squares, a small white one, a small black one, and a large yellow one. The small white square is touching the left edge of C, while the small black square is touching the right edge of C. The large yellow square is adjacent to the white square, and is touching the bottom edge of C. 
question 1: Is a circle below a rectangle? 
question 2: Is the red circle above the white square?
question 3 : Which block has only circles inside? 
question 4: What block contains all of the squares?
quesiton 5: Which object is to the left of the yellow square? the white square or the black square? 
question 6: What object is not above the purple rectangle? the orange triangle or the white square?
question 7: What is the relation between the black squares and yellow squares?
question 8: What is the relation between the red circle and the yellow square?

[Ouptput]
block(a;b;c).
object(small_red_circle, a).
object(green_circle, a).
object(big_blue_circle, a).
object(orange_triangle, b).
object(purple_rectangle, b).
object(small_white_square, c).
object(small_black_square, c).
object(large_yellow_square, c).
is(small_red_circle, above, big_blue_circle).
is(green_circle, above, big_blue_circle).
is(big_blue_circle, touching, bottom_edge_a).  % Representing touching the bottom edge
is(green_circle, touching, top_edge_a).  % Representing touching the top edge
is(big_blue_circle, below, green_circle).  % This is inferred the touching information in block a
is(orange_triangle, above, purple_rectangle).
is(purple_rectangle, touching, bottom_edge_b).  % Representing touching the bottom edge
is(small_white_square, touching, left_edge_c).  % Representing touching the left edge
is(small_black_square, touching, right_edge_c).  % Representing touching the right edge
is(small_black_square, right, small_white_square). % This is inferred the touching information in block c
is(large_yellow_square, adjacent, small_white_square).  % Representing adjacency
is(large_yellow_square, touching, bottom_c).
is(a, top, b).
is(b, top, c).

% Question 1: Is a circle below a rectangle?
query(yes) :- object(Circle, BlockC), object(Rectangle, BlockR),
              Circle = small_red_circle; Circle = green_circle; Circle = big_blue_circle,
              Rectangle = purple_rectangle,
              is(Rectangle, above, Circle).
query(no) :- not query(yes).

% Question 2: Is the red circle above the white square?
query(yes) :- is(small_red_circle, above, small_white_square).
query(no) :- not query(yes).

% Question 3: Which block has only circles inside?
query(Block) :- block(Block), object(X, Block),
                X= small_red_circle; X = green_circle; X = big_blue_circle.
% Question 4: What block contains all of the squares?
query(Block) :- block(Block),
                object(small_white_square, Block),
                object(small_black_square, Block),
                object(large_yellow_square, Block).

% Question 5: Which object is to the left of the yellow square? the white square or the black square?
query(LeftObject) :- is(large_yellow_square, left, LeftObject), LeftObject = small_white_square; LeftObject = small_black_square.

% Question 6: What object is not above the purple rectangle? the orange triangle or the white square?
query(NotAboveObject) :- not is(NotAboveObject, above, purple_rectangle), NotAboveObject = orange_triangle; NotAboveObject = small_white_square.

% Question 7: What is the relation between the black squares and yellow squares?
query(Relation) :- is(small_black_square, Relation, large_yellow_square).

% Question 8: What is the relation between the red circle and the yellow square?
query(Relation) :- is(small_red_circle, Relation, large_yellow_square).

## Step 3: Review Checklist

Before outputting the final version, verify the following:

1. Predicate usage: Confirm correct use of object/2, block/1, is/3.
2. Number of arguments: Ensure object/2 and is/3 have the correct number of arguments and in the right order.
3. Spatial relations: Use only standard spatial relations in is/3 predicates.
4. Consistent naming: Ensure object identifiers are consistent across facts and queries.
5. Query format: Use appropriate query format based on the question type (Yes/No or Wh-question).
6. Variable consistency: Confirm variables in the query head appear in the query body.
7. Lowercase usage: Use lowercase for all names and variables.
8. Prolog syntax: Verify correct use of Prolog-specific syntax.
9. Punctuation and formatting: Check for correct use of periods, commas, parentheses, and new lines.

[expected output format]
% Facts
<refined facts >
% Query
<query(yes):- ...>
<query(no):- ...>
<query(Answer):- ...>
#show query/1.
## Remove any comments or code block markers. Output only the final refined version of facts and query.
"""

refine_prompt= """ Given the facts and query, refine the ASP code to ensure correctness and executability. Follow these steps:
1. Add Standard Rules:
   Append these rules after the facts and query:
   % Inverse relations
   inverse(left, right; right, left; front, behind; behind, front; above, below; below, above; near, far; far, near; in, contain; contain, in).
   is(Y, R2, X) :- is(X, R1, Y), inverse(R1, R2).
   % Symmetric relations
   symmetric(near; far; touching; disconnected; overlapping; adjacent).
   is(Y, R, X) :- is(X, R, Y), symmetric(R).
   % Transitive relations
   transitive(above; below; left; right; front; behind; far; contain; in).
   is(X, R, Z) :- is(X, R, Y), is(Y, R, Z), transitive(R), X != Y, Y != Z, X != Z.
   % Inter-block object relations
   is(O1, R, O2) :- object(O1, _, _, _, B1), object(O2, _, _, _, B2), is(B1, R, B2), O1 != O2.
   % Transitive inter-block object relations
   is(O1, R, O3) :- object(O1, _, _, _, B1), object(O2, _, _, _, B2), object(O3, _, _, _, B3),
                    is(B1, R, B2), is(B2, R, B3), transitive(R),
                    B1 != B2, B2 != B3, B1 != B3, O1 != O2, O2 != O3, O1 != O3.
2. Review and Refine:
   a) Facts:
      - Use only is/3, block/1, object/5 as predicates
      - Ensure consistent naming of arguments and relations
      - Map relation synonyms to: left, right, above, below, near, far, front, behind, in, contain, touching, disconnected, overlapping
      - Verify unique and consistent object identifiers
   b) Query:
      - Use query(yes), query(no), or query(answer) based on question type
      - Ensure consistency with facts and question conditions
      - Verify all necessary facts and rules are present
      - Check for variable safety and alignment
   c) Syntax:
      - Use "unknown" for unspecified properties in object/5
      - Verify correct use of periods, commas, parentheses
      - End each fact, rule, and query with a period
      - Maintain naming consistency across facts and queries
3. Error Identification and Fixes:
   a) Syntax Errors:
      - "syntax error, unexpected ., expecting :-" -> Fix rule structure (head :- body.)
      - "syntax error, unexpected identifier" -> Check predicate/argument spelling
   b) Unsafe Variables:
      - "error: unsafe variables in: q(X) :- p(Y)." -> Bind all variables in positive literals
   c) Grounding Errors:
      - "error: function symbol expected" -> Use correct number of arguments
   d) Undefined Predicates:
      - "info: atom does not occur in any rule head: p(X)" -> Define all used predicates
   e) Inconsistent Rules:
      - "error: inconsistent rules:" -> Review and resolve conflicts
   f) Show Directive Errors:
      - "error: unsafe variables in: #show p(X)." -> Ensure bound variables in #show directives
4. Final Output:
   Provide only the refined ASP code in this structure:
   % Facts
   <refined facts>
   % Rules
   <refined rules>
   % Query
   <refined query>
   #show query/1.

Ensure the final ASP program is 100% executable, free of errors, and accurately represents the given scenario."""

print(len(refine_prompt))

prompt_facts= """[Role] You are an expert parser, specializing in converting natural language descriptions into precise, logical representations. 
Step 1: Sentence-Facts Conversion:
- Identify ALL objects, block and Relation FACTS in EVERY SENTENCE of the context. Sometimes there are multiple facts in a sentence, sometimes the relation need to be inferred, so you need to identify all of them.
- Represent every sentence as facts using ONLY these THREE predicates (object/5, is/3 and block/1):
   a) block(name), eg. block(a). block(a;b;c).
   b) object(distinct_name, size, color, shape, block_located).
        eg. object(large_blue_triangle, large, blue, triangle, a). object(blue_oval, unknown, blue, oval, b).
   c) is(A, relation, B), eg. is(large_blue_triangle, left_of, blue_oval), is(a, left, b).  
   
[Critical Rules to follow] 
- use of "unknown" for unspecified properties instead of placeholder "_" to avoid the unsafe variable error. Keep the order of arguments and separate them with comma.
- For the is(A, relation, B) facts, the relation must be one of the following ten relations: left, right, above, below, near, far, front, behind, adjacent, disconnected. Do not use any other relation names such as "touching," "left_far," or "right_front."
eg. Incorrect: is(medium_blue_square, far_left, yellow_square) Correct: is(medium_blue_square, far, yellow_square). is(medium_blue_square, left, yellow_square).
- If the context does not explicitly state a relationship between two objects, do infer any relationship from the touching or adjacent information.
   eg.“The big blue circle is touching the bottom edge of A, and the green circle is touching the top edge of A.” should be represented as is(green_circle, above, big_blue_circle).
  Instead of representing the touching relation explicitly, YOU Should nfer the spatial relationship between objects in the same block from their relative touching edges on the block.

Step 2: Question to Query Conversion: 
Query format: query(1 argument):- translation of facts and condition in the question using object/5, is/3 and block/1 predicates.
Yes/No questions: Use query(yes) and query(no).
Wh-questions: query(Variable) :- conditions, Variable = result.

[Critical Rules to follow]  
- Pay attention "all" and "only" quantifiers and Ensure that the query accounts for all relevant entities and relationships.
- Use negation for "no" or "not any" quantifiers.
- Ensure that all variables in the query's head (left side) appear in the body (right side). This ensures the query is "safe" and syntactically correct.
- Verify that the query predicates match the same names, relations, and objects used in the facts.

- "All":  Use negation to find exceptions. For Yes/No question, query(no) to indicate that there is at least one fact that violates the condition.
% Question:  Are all squares in B?
query(no) :- object(_, _, _, square, Block), Block != b.  query(yes) :- not query(no).
Another method is to use conditional literals (:)  instead of "," to represent "for all".
% Are all objects in front of the black object?"
   query(yes) :- object(B, _, black, _, _), is(O, front, B) : object(O, _, _, _, _), O != B. query(no) :- not query(yes). 
- "Only": Check for absence elsewhere. E.g., "Are X only in Y?":
query(no) :- object(_, _, _, X, Block), Block != Y. query(yes) :- not query(no).

% What is the relation between the small blue thing and the large blue thing?
query(Relation) :-  object(SBT, small, blue, _, _),   object(LBT, large, blue, _, _), is(SBT, Relation, SBT).
% Where is the circle left to a square regarding to a circle below a square?
query(Relation) :- object(circle1, _, _, circle, _), object(square, _, _, square, _), object(circle2 _, _, circle, _), is(circle1, left, square), is(circle2, below, square), is(circle1,Relation,circle2).
E.g. What block has all of the large objects inside?
query(Block) :- block(Block), not object(_, large, _, _, OtherBlock): block(OtherBlock), OtherBlock != Block. % conditional literals (:)  represent "for all".
- "Only": Check for absence elsewhere.
eg. Which block has only squares inside?"
query(Block) :- block(Block), object(_, _, _, circle, Block),not object(_, _, _, shape, Block), shape  !=circle.
% Question : What object is not near the large yellow thing?
   query(Object) :- object(LargeYellow, large, yellow, _, _),object(Object,_,_,_,_),not is(Object, near, LargeYellow).
What is above the small yellow triangle? a big triangle  or  a blue triangle?
  query(Object) :- object(Object _, _, _ , _ ),object(SmallYT, small,yellow , triangle, _ ), is(Object, above, SmallYT).

Step 3. Review before the output the final version:

Verify predicate usage: object/5, block/1, is/3, especially, the order of arguments in object/5.
Use only standard spatial relations in is/3
Ensure consistent object identifier naming across facts and query
Use appropriate query format based on question type
Confirm variables in query head appear in query body
Use lowercase for all names and variables
Verify correct use of Prolog-specific syntax.
Check for correct use of periods, commas, period, and parentheses.

[expected output format]
% Facts
<refined facts >
% Query
<refined query>

Remove any comments or code block markers. Output only the final refined version of facts and query."""


3060


In [52]:
complete_asp= """ [Role] You are an expert parser, specializing in converting natural language descriptions into precise, logical representations. 
Step 1: Sentence-Facts Conversion:
- Identify ALL Relation FACTS in EVERY SENTENCE of the context. Sometimes there are multiple facts in a sentence, so you need to identify all of them.
- Represent every sentence as facts using ONLY these THREE predicates (object/5, is/3 and block/1):
   a) block(name), eg. block(a). block(a;b;c).
   b) object(distinct_name, size, color, shape, block_located), 
        eg. object(large_blue_triangle, large, blue, triangle, a). object(blue_oval, unknown, blue, oval, b).
   c) is(A, relation, B), eg. is(large_blue_triangle, left_of, blue_oval), is(a, left, b).  
   
[Critical Rules to follow] 
- is(A,relation,B) should include all the relations facts mentioned in the context and question, including relation between blocks, the relation between objects within blocks, relations objects between blocks. 
- Relation must be one of the following: left, right, above, below, near, far, front, behind, in, contain, touching, disconnected, overlapping.
- Use "unknown" for unspecified properties in object(5 arguments). Keep the order of arguments,and separate arguments with commas. 

Examples:
[Context]: There is a block A below block B. In A, there is a large blue triangle to the left of and below a large black square. Far above the square is a large black circle.
[Facts]: block(a;b). is(a, below, b).object(large_black_square, large, black, square, a). object(large_blue_triangle, large, blue, triangle, a). is(large_blue_triangle, left, large_black_square). is(large_blue_triangle, below, large_black_square). is(large_black_circle, far, large_black_square),is(large_black_circle, above, large_black_square).    
[Context]: Block BBB is over and behind block AAA. Disconnected from and farther from this block there is block AAA with a medium yellow square.
[Facts]: is(bbb, over, aaa). is(bbb, behind, aaa). is(bbb, disconnected, aaa). is(bbb, far, aaa). object(yellow_medium_square, medium, yellow, square, aaa).

Step 2: Question to Query Conversion: 
Query format: query(1 argument):- translation of facts and condition in the question using object/5, is/3 and block/1 predicates.
For Yes/No questions: Use query(yes) and query(no).
For Wh-Questions: Use query(relation), query(object), or query(block) as appropriate.

[Critical Rules to follow]  
- Pay attention "all" and "only" quantifiers and Ensure that the query accounts for all relevant entities and relationships.
- Use negation for "no" or "not any" quantifiers.
- Ensure that all variables in the query's head (left side) appear in the body (right side). This ensures the query is "safe" and syntactically correct.
- Verify that the query predicates match the same names, relations, and objects used in the facts.

Examples:
% Does B have any circles?
query(yes) :- block(b), object(_, _, _, circle, b). query(no) :- not query(yes).
%hint: you can use query(no) to indicate that there is at least one fact that violates the condition.
% Query: Are all squares in B?
query(no) :- object(_, _, _, square, Block), Block != b. query(yes) :- not query(no).
% Query: Does A have only triangles?
query(no) :- object(_, _, _, Shape, a), Shape != triangle. query(yes) :- not query(no).
% Question: Are all yellow squares in the same block?
query(yes) :- 1 = #count{Block : object(_, _, yellow, square, Block)}.  query(no) :- not query(yes).

% Question: What is the relation between the small black triangle in B and the small blue thing in c?
query(Relation) :- object(small_black_triangle, small, black, triangle, b), object(small_blue_thing, small, blue, thing, c), is(small_black_triangle, Relation, small_blue_thing).
% Question : What object is not near the large yellow thing?
   query(Object) :- object(LargeYellow, large, yellow, _, _),object(Object,_,_,_,_),not is(Object, near, LargeYellow).
% Question:What block does not have a triangle inside?
  query(Block) :- block(Block), not object(_, _, _, triangle, Block).
  
[Step 3: Rule Adaptation]
Add the following rules:
% Inverse relations
inverse(left, right; right, left; front, behind; behind, front; above, below; below, above; near, far; far, near; in, contain; contain, in).
is(Y, R2, X) :- is(X, R1, Y), inverse(R1, R2).
% Symmetric relations
symmetric(near; far; touching; disconnected; overlapping; adjacent).
is(Y, R, X) :- is(X, R, Y), symmetric(R).
% Transitive relations
transitive(above; below; left; right; front; behind; far; contain; in).
is(X, R, Z) :- is(X, R, Y), is(Y, R, Z), transitive(R), X != Y, Y != Z, X != Z.
% Inter-block object relations
is(O1, R, O2) :- object(O1, _, _, _, B1), object(O2, _, _, _, B2), is(B1, R, B2), O1 != O2.
% Transitive inter-block object relations
is(O1, R, O3) :- object(O1, _, _, _, B1), object(O2, _, _, _, B2), object(O3, _, _, _, B3),
                 is(B1, R, B2), is(B2, R, B3), transitive(R),
                 B1 != B2, B2 != B3, B1 != B3, O1 != O2, O2 != O3, O1 != O3.

[Step 4: ASP refinenment]
1. Check facts:
   - Use only is/3, block/1, object/5 as predicates
   - Use the name of arguments and valid relation names consistently in the facts and query
   - Map relation synonyms to the relation set: left, right, above, below, near, far, front, behind, in, contain, touching, disconnected, overlapping.
   - Ensure unique object identifiers and consitent across the facts and query.
2. Verify Query 
  - Use query(yes), query(no), or query(answer) appropriately based on the question type.
  - Ensure that the query is consistent with the facts and conditions in the question.
  - Cross-check the query with both the facts and the question to ensure that the conversion accurately reflects what is being asked.
  - Ensure no variables or relationships are left out or incorrectly represented.
  - Verify that all necessary facts and rules are present to support the query.
  - Ensure all variables in head appear in the bodies (safety and alignment)
3. Check syntax:
   - Ensure variable safety, eg. Use "unknown" for unspecified properties in object/5 in the facts
   - Verify correct use of periods, commas, parentheses
   - End each fact, rules and query with stop.
   - Maintain consistency in naming and arguments across facts and queries.
   - Verify correct use of Prolog-specific syntax

[Step 5: Final Output]
   - Output only the facts,query and rules.
   - Remove any comments, unnecessary spaces, new lines and punctuation marks
   - Delele 'prolog', 'asp', code block markers, like ```, **, etc to keep the code clean and runnable by solver

[Final ASP Program Structure]
% Facts
<all extracted facts>
% Query
<question converted query>
#show query/1.
% Rules
<selected relevant rules>
"""
print(len(complete_asp))




6760


In [10]:
import dspy
import pandas as pd
import json
import re
import sys
from io import StringIO
from typing import List, Dict, Any
from dataclasses import asdict
import clingo

class Convert(dspy.Module):
    class Signature(dspy.Signature):
        prompt_1 = dspy.InputField(desc="The prompt for the task")
        context = dspy.InputField(desc="Spatial description of scene, to be parsed into facts")
        question = dspy.InputField(desc="The question to be parsed into query")
        asp = dspy.OutputField(desc="the complete and refined asp program")

    def __init__(self, state):
        super().__init__()
        self.state = state
        self.predictor = dspy.ChainOfThought(self.Signature)

    def forward(self, prompt_1, context, question):
        result = self.predictor(prompt_1=prompt_1, context=context, question=question)
        asp = result.asp
        self.state['convert'] = {
            'context': context,
            'question': question,
            'prompt_1': prompt_1,
            'asp': asp,
        }
        return dspy.Prediction(asp=asp)
    
class ReviseASP(dspy.Module):
    class Signature(dspy.Signature):
        """
[Task] Revise the ASP program to ensure it is syntactically correct, logically sound, and capable of successful grounding and query satisfaction in Clingo.
1. Handling Parsing Errors:
End with Periods: Ensure every fact and rule ends with a period (.).
Predicate Structure: Verify correct use of predicates with the appropriate number of arguments (e.g., object/5, is/3).
Variable Consistency: Ensure all variables in rule heads appear in rule bodies to maintain safety.
Correct Syntax: Check for proper use of commas, periods, and parentheses in rules and facts.
2. Handling Grounding Errors:
Complete Object Definitions: Fully define all objects with the necessary attributes, using "unknown" for unspecified properties.
Check Rule Consistency: Ensure rules are free from circular dependencies and logically follow from the facts.
Validate Predicate Arguments: Ensure predicates have the correct number of arguments and align with defined facts.
3. Handling "Satisfiable but No Query Results":
Align Query with Facts: Ensure the query components match the facts. The query should be directly supported by existing facts.
Review and Adjust Rules: Revise rules to ensure they logically support the query and cover all relevant relationships.
Check Query Feasibility: Ensure the query can be satisfied based on the facts and rules; revise as needed.
[Final ASP Program Structure]
% Facts
<all extracted facts>
% Rules
<selected relevant rules>
% Query
<question converted query>
        """
        asp= dspy.InputField(desc="Initial ASP facts, query and rules ")
        feedback= dspy.InputField(desc="Feedback for Clingo Running Errors")
        asp_code = dspy.OutputField(desc="Revised ASP code, keep it clean and correct. Limit your output to 3000 tokens")

    def __init__(self, state):
        super().__init__()
        self.state = state
        self.predictor = dspy.ChainOfThought(self.Signature)

    def forward(self, asp, prompt_1,context, question,feedback):
        # Retrieve facts, context_question, and prompt_1 from shared state
        convert_data = self.state.get('convert', {})
        context = convert_data.get('context', '')
        question= convert_data.get('question', '')
        prompt_1 = convert_data.get('prompt_1', '')
        result = self.predictor(asp=asp, prompt_1 =prompt_1,context=context, question=question,feedback= feedback)
        asp_code = result.asp_code
        self.state['asp'] = {
            'asp_code': asp_code,
        }
        return dspy.Prediction(asp_code=asp_code)

class RunClingo(dspy.Module):
    class Signature(dspy.Signature):
        asp_code = dspy.InputField(desc="Initial ASP code to be run and evaluated")
        result = dspy.OutputField(desc="Result from Clingo ASP Solver")
        error = dspy.OutputField(desc="Error output from Clingo ASP Solver")

    def __init__(self, state):
        super().__init__()
        self.state = state
        self.predictor = dspy.Predict(self.Signature)

    def forward(self, asp_code):
        result, error = self.run_clingo(asp_code)
        self.state['running_clingo'] = {
            'result': result,
            "error": error,
        }
        return dspy.Prediction(result=result, error=error)
    
    def run_clingo(self, asp_code):
        results = []
        error_output = StringIO()
        # clean the asp code
        asp_code = asp_code.replace("prolog", "").replace("```", "").replace("\n\n\nQuery:","").strip()
    
        try:
            # Redirect stderr to capture errors
            original_stderr = sys.stderr
            sys.stderr = error_output

            # Initialize Clingo control
            ctl = clingo.Control(["0", "--warn=all"])
            ctl.add("base", [], asp_code)
            ctl.ground([("base", [])])

            # Define on_model callback to collect query results
            def on_model(model):
                for atom in model.symbols(shown=True):
                    if atom.name == "query" and atom.arguments:
                        results.append(str(atom.arguments[0]))

            # Solve and collect results
            solve_result = ctl.solve(on_model=on_model)

            # Process the results
            if solve_result.satisfiable:
                asp_output = ", ".join(results) if results else "satisfiable, but no query results"
            elif solve_result.unsatisfiable:
                asp_output = "unsatisfiable"
            else:
                asp_output = "unknown"

        except Exception as e:
            asp_output = f"Error: {str(e)}"
        finally:
            sys.stderr = original_stderr

        error_string = error_output.getvalue()
        error_output.close()

        return asp_output, error_string
 
class Pipeline(dspy.Module):
    def __init__(self, state, max_iters=4):
        super().__init__()
        self.state = state
        self.convert = Convert(state)
        self.clingo = RunClingo(state)  # Initialize RunClingo with state
        self.revise = ReviseASP(state)
        self.max_iters = max_iters
        
    def forward(self, context, question, prompt_1):
        # Convert the natural language description into ASP facts and query
        convert_result = self.convert.forward(prompt_1=prompt_1, context=context, question=question)
        asp = convert_result.asp
        asp_code = asp
        for _ in range(self.max_iters):
            clingo_result = self.clingo.forward(asp_code=asp_code)
            # Check for errors and update feedback if necessary
            if "error" not in clingo_result.result.lower():
                return dspy.Prediction(asp_code=asp_code, result=clingo_result.result, error=clingo_result.error)
            revise_asp_result= self.revise(asp=asp,prompt_1=prompt_1, context=context, question=question,feedback=clingo_result.result)
            asp_code = revise_asp_result.asp_code

        return dspy.Prediction(asp_code=asp_code,result= clingo_result.result, error=clingo_result.error)
    
def process_examples(examples: List[dspy.Example], pipeline: Pipeline) -> List[Dict[str, Any]]:
    results = []
    for example in examples:
        try:
            context = example.get('context')
            question = example.get('question')
            prompt_1 = example.get('prompt_1')
            q_type=  example.get('q_type')
            prediction = pipeline(context= context, question=question, prompt_1=prompt_1)
           
            result = {
            "context": context,
            "question": question,
            "q_type": q_type,
            "choices":example.get('choices'),
            "predicted_ASP": prediction.asp_code,
            "asp_result": prediction.result,
            "error": prediction.error,
            "actual_answer": example.get('answer')}
            results.append(result)
     
        except Exception as e:
            print(f"Error processing example: {example.question}")
            print(f"Error: {str(e)}")
    return results

def main(input_file: str, output_file: str):
    df = pd.read_csv(input_file)
    clean_data = df.to_dict(orient='records')

    examples = [
        dspy.Example(
            prompt_1= new_prompt_facts,
            context=r["Story"],
            q_type=r["Q_type"],
            question="".join(r["Question"]),
            choices="".join(r["Candidate_Answers"]),
            answer="".join(r["Answer"])
        ).with_inputs("context", "question", "prompt_1", "choices")
          for r in clean_data 
           if  not pd.isna(r["Reasoning_type"]) and "Quantifier" in ast.literal_eval(r["Reasoning_type"])
    ]
    print(len(examples))
    state= {}
    pipeline = Pipeline(state)
    results = process_examples(examples, pipeline)

    with open(output_file, "w") as jsonfile:
        json.dump(results, jsonfile, indent=4)

if __name__ == "__main__":
    main("Spar_test.csv", "Spar_human/llama3_quantifer_5_iterations.json")

36


<block>:1:6-8: error: syntax error, unexpected <IDENTIFIER>

<block>:65:5-6: error: syntax error, unexpected :

<block>:66:5-6: error: syntax error, unexpected :

<block>:65:5-6: error: syntax error, unexpected :

<block>:1:6-8: error: syntax error, unexpected <IDENTIFIER>

<block>:65:6-10: error: syntax error, unexpected <IDENTIFIER>

<block>:66:6-10: error: syntax error, unexpected <IDENTIFIER>

<block>:65:6-10: error: syntax error, unexpected <IDENTIFIER>

<block>:1:6-8: error: syntax error, unexpected <IDENTIFIER>

<block>:82:6-10: error: syntax error, unexpected <IDENTIFIER>

<block>:75:1-74: error: unsafe variables in:
  query(yes):-[#inc_base];block(b);Block!=b;not #p_object(#p,#p,#p,triangle,#b(Block)).
<block>:75:63-68: note: 'Block' is unsafe

<block>:78:6-13: error: syntax error, unexpected <IDENTIFIER>

<block>:1:6-8: error: syntax error, unexpected <IDENTIFIER>

<block>:73:5-6: error: syntax error, unexpected :

<block>:74:5-6: error: syntax error, unexpected :

<block>:74

In [87]:
import json
import xml.etree.ElementTree as ET

# Function to convert JSON object to XML element
def json_to_xml(obj, root):
    if isinstance(obj, dict):
        for key, value in obj.items():
            child = ET.SubElement(root, key)
            json_to_xml(value, child)
    elif isinstance(obj, list):
        for item in obj:
            child = ET.SubElement(root, "item")
            json_to_xml(item, child)
    else:
        root.text = str(obj)

# Load JSON data from file
with open('Spar_human/deepseek_quantifier_1_iteration.json', 'r') as json_file:
    data = json.load(json_file)

# Create the root element
root = ET.Element("root")

# Convert JSON to XML
json_to_xml(data, root)

# Create an XML tree and write to a file
tree = ET.ElementTree(root)
tree.write("Spar_human/deepseek_quantifier_1_iteration.xml", encoding='utf-8', xml_declaration=True)

In [156]:
import dspy
import pandas as pd
import json
import re
import sys
from io import StringIO
from typing import List, Dict, Any
from dataclasses import asdict
import clingo

class Convert(dspy.Module):
    class Signature(dspy.Signature):
        prompt_1 = dspy.InputField(desc="The prompt for the task")
        context = dspy.InputField(desc="Spatial description of scene, to be parsed into facts")
        question = dspy.InputField(desc="The question to be parsed into query")
        facts = dspy.OutputField(desc="The initial facts and query")

    def __init__(self, state):
        super().__init__()
        self.state = state
        self.predictor = dspy.ChainOfThought(self.Signature)

    def forward(self, prompt_1, context, question):
        result = self.predictor(prompt_1=prompt_1, context=context, question=question)
        facts = result.facts
        self.state['convert'] = {
            'facts': facts,
            'context': context,
            'question': question,
        }
        return dspy.Prediction(facts=facts)

class RunClingo(dspy.Module):
    class Signature(dspy.Signature):
        """Run Clingo and capture feedback."""
        facts = dspy.InputField(desc="Initial ASP code to be evaluated")
        result = dspy.OutputField(desc="Result from Clingo")
        error = dspy.OutputField(desc="Error output from Clingo")

    def __init__(self, state):
        super().__init__()
        self.state = state
        self.predictor = dspy.Predict(self.Signature)

    def forward(self, facts):
        result, error = self.run_clingo(facts)
        return dspy.Prediction(result=result, error=error)
    
    def run_clingo(self, facts):
        results = []
        error_output = StringIO()
        
        # Extract the ASP program from the input facts
        match = re.search(r'block(.*?)#show query/1\.', facts, re.DOTALL)
        asp_program = match.group(1).strip() if match else ""
        asp_program = f"block{asp_program}\n#show query/1."

        try:
            original_stderr = sys.stderr
            sys.stderr = error_output
            
            ctl = clingo.Control()
            ctl.add("base", [], asp_program)
            ctl.ground([("base", [])])

            def on_model(model):
                for atom in model.symbols(shown=True):
                    if atom.name == "query" and atom.arguments:
                        results.append(str(atom.arguments[0]))

            solve_result = ctl.solve(on_model=on_model)

            if solve_result.satisfiable:
                output = "Answer: " + ", ".join(results) if results else "satisfiable, but no query results"
            elif solve_result.unsatisfiable:
                output = "unsatisfiable"
            else:
                output = "Unknown"

        except Exception as e:
            output = f"Error: {str(e)}"
        finally:
            sys.stderr = original_stderr

        error_string = error_output.getvalue()
        error_output.close()

        return output, error_string
    
class ReviseASP(dspy.Module):
    class Signature(dspy.Signature):
        """Revise and refine ASP code based on Clingo feedback."""
        facts = dspy.InputField(desc="Initial facts and query")
        prompt_2 = dspy.InputField(desc="The instructions to refine ASP code")
        #feedback = dspy.InputField(desc="Feedback from Clingo")
        asp_code = dspy.OutputField(desc="The refined complete ASP program. Limit your output to 3000 tokens")

    def __init__(self, state):
        super().__init__()
        self.state = state
        self.predictor = dspy.ChainOfThought(self.Signature)

    def forward(self, facts, prompt_2):# feedback
        result = self.predictor(facts=facts,  prompt_2=prompt_2)#feedback=feedback,
        asp_code = result.asp_code
        self.state['asp'] = {
            'asp_code': asp_code,
        }
        return dspy.Prediction(asp_code=asp_code)

class Pipeline(dspy.Module):
    def __init__(self, state, max_iters=2):
        super().__init__()
        self.state = state
        self.convert = Convert(state)
        self.asp = ReviseASP(state)
        #self.clingo = RunClingo(state)  # Initialize RunClingo with state
        self.max_iters = max_iters

    def forward(self, context, question, prompt_1, prompt_2):# feedback=""
        # Convert the natural language description into ASP facts and query
        convert_result = self.convert.forward(prompt_1=prompt_1, context=context, question=question)
        facts = convert_result.facts
        asp_code = None
        for _ in range(self.max_iters):
            revise_result = self.asp.forward(facts=facts, prompt_2=prompt_2) #feedback=feedback
            asp_code = revise_result.asp_code
            
            # Run Clingo on the revised ASP code
            #clingo_result = self.clingo.forward(facts=asp_code)
            # Check for errors and update feedback if necessary
            # if clingo_result.error:
            #     feedback = clingo_result.error  # Update feedback with Clingo errors
            #     break  # Exit if there's an error

        return dspy.Prediction(asp_code=asp_code)# error=clingo_result.error

def process_examples(examples: List[dspy.Example], pipeline: Pipeline) -> List[Dict[str, Any]]:
    results = []
    for example in examples:
        try:
            context = example.get('context')
            question = example.get('question')
            prompt_1 = example.get('prompt_1')
            prompt_2 = example.get('prompt_2')
            q_type=  example.get('q_type')
            prediction = pipeline(context=example.context, question=example.question, prompt_1=example.prompt_1, prompt_2=example.prompt_2)
           
            result = {
            "context": context,
            "question": question,
            "q_type": q_type,
            "choices":example.get('choices'),
            "predicted_ASP": prediction.asp_code,
            "actual_answer": example.get('answer')}
            results.append(result)
     
        except Exception as e:
            print(f"Error processing example: {example.question}")
            print(f"Error: {str(e)}")
    return results

def main(input_file: str, output_file: str):
    df = pd.read_csv(input_file)
    sampled_df = df.head(10).copy()
    clean_data = sampled_df.to_dict(orient='records')

    examples = [
        dspy.Example(
            prompt_1=prompt_ASP,
            prompt_2=prompt_Refine,
            context=r["Story"],
            q_type=r["Q_type"],
            question="".join(r["Question"]),
            choices="".join(r["Candidate_Answers"]),
            answer="".join(r["Answer"])
        ).with_inputs("context", "question", "prompt_1", "choices", "prompt_2", "q_type")
        for r in clean_data  if r["Q_type"] == "YN" and "Quantifier" in r["Reasoning_type"]
    ]

    state = {}
    pipeline = Pipeline(state)
    results = process_examples(examples, pipeline)

    with open(output_file, "w") as jsonfile:
        json.dump(results, jsonfile, indent=4)

if __name__ == "__main__":
    main("Spar_test.csv", "llama3_ASP_first_10.json")


INFO:backoff:Backing off request(...) for 0.2s (groq.InternalServerError: Error code: 500 - {'error': {'message': 'Internal Server Error', 'type': 'internal_server_error'}})


Backing off 0.2 seconds after 1 tries calling function <function GroqLM.request at 0x128167240> with kwargs {'max_tokens': 2000, 'n': 1, 'temperature': 0.0}


In [4]:
import json
import re
import sys
from io import StringIO
import clingo

def extract_asp_program(predicted_asp):
    # Find the ASP program between "% Facts" and "#show query/1."
    match = re.search(r'block(.*?)#show query/1\.', predicted_asp, re.DOTALL)
    if match:
        asp_program = match.group(1).strip()
        # Add back the start and end markers to the extracted program
        asp_program = "block" + asp_program + "\n#show query/1."
        # asp_program = asp_program.replace('\n', ' ')
     
        # # Remove extra whitespace
        # asp_program = ' '.join(asp_program.split())
        return asp_program
    return None
 
def run_asp_and_get_answer(asp_program):
    results = []
    error_output = StringIO()
    
    try:
        # Redirect stderr to capture errors
        original_stderr = sys.stderr
        sys.stderr = error_output

        # Initialize Clingo control
        ctl = clingo.Control()
        ctl.add("base", [], asp_program)
        ctl.ground([("base", [])])

        # Define on_model callback to collect query results
        def on_model(model):
            for atom in model.symbols(shown=True):
                if atom.name == "query" and atom.arguments:
                    results.append(str(atom.arguments[0]))

        # Solve and collect results
        solve_result = ctl.solve(on_model=on_model)

        # Process the results
        if solve_result.satisfiable:
            asp_output = ", ".join(results) if results else "satisfiable, but no query results"
        elif solve_result.unsatisfiable:
            asp_output = "unsatisfiable"
        else:
            asp_output = "unknown"

    except Exception as e:
        asp_output = f"Error: {str(e)}"
    finally:
        sys.stderr = original_stderr

    error_string = error_output.getvalue()
    error_output.close()

    return asp_output, error_string 

def process_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    for item in data:
        predicted_asp = item.get('predicted_ASP', '')
        asp_program = extract_asp_program(predicted_asp)
        asp_result, _ = run_asp_and_get_answer(asp_program)
        
        item['clean_asp'] = asp_program
        item['asp_result'] = asp_result

    return data

# Test the function
file_path = "deepseek_ASP_last_50_processed.json"
processed_data = process_json_file(file_path)

# Print the first result to check
print(json.dumps(processed_data[4], indent=2))

# Optionally, save the processed data back to a file
with open("deepseek_ASP_last_50_processed.json", 'w') as file:
    json.dump(processed_data, file, indent=2)

{
  "context": "There are three blocks called A, B, and C. A is to the left of B, and B is to the left of C. In A, there is a large blue triangle to the left of and below a large black square. Far above the square is a large black circle. In B, there is a large blue circle near and below a small blue circle. The small blue circle is near and below a large yellow square. There is a medium blue square far to the left of the yellow square. In C, there is a large blue square.",
  "question": "Is the large yellow thing above a medium blue square?",
  "q_type": "YN",
  "choices": "['Yes', 'No', 'DK']",
  "predicted_ASP": "Here is the refined complete ASP program:\n\n```\n% Facts\nblock(a;b;c).\nis(a, left, b).\nis(b, left, c).\nobject(large_blue_triangle, large, blue, triangle, a).\nobject(large_black_square, large, black, square, a).\nis(large_blue_triangle, left, large_black_square).\nis(large_blue_triangle, below, large_black_square).\nobject(large_black_circle, large, black, circle, a).\

<block>:38:246-247: error: syntax error, unexpected ","

<block>:33:111-160: info: atom does not occur in any rule head:
  is(LargeBlackSquare,far,below,OtherLargeBlack)

<block>:33:116-117: error: syntax error, unexpected =, expecting )

<block>:34:140-141: error: syntax error, unexpected ","

<block>:10:1-47: error: unsafe variables in:
  object(blue_thing,unknown,blue,unknown,#Anon0):-[#inc_base].
<block>:10:44-45: note: '#Anon0' is unsafe

<block>:22:92-93: error: syntax error, unexpected =, expecting )

<block>:23:133-134: error: syntax error, unexpected =, expecting )

<block>:31:1-15: info: no atoms over signature occur in program:
  query/1

<block>:3:1-42: error: unsafe variables in:
  object(big_blue_shape1,big,blue,#Anon0,a):-[#inc_base].
<block>:3:36-37: note: '#Anon0' is unsafe

<block>:4:1-42: error: unsafe variables in:
  object(big_blue_shape2,big,blue,#Anon0,a):-[#inc_base].
<block>:4:36-37: note: '#Anon0' is unsafe

<block>:7:1-49: error: unsafe variables in:
  object

In [ ]:
import dspy
import pandas as pd
import json
import re
from io import StringIO
from typing import List, Dict, Any
from dataclasses import asdict
import sys
import ast
import clingo


class Convert(dspy.Module):
    class Signature(dspy.Signature):
        prompt_1 = dspy.InputField(desc="The prompt for the task")
        context = dspy.InputField(desc="Spatial description of scene to be parsed into facts")
        question = dspy.InputField(desc="The question to be parsed into query")
        asp = dspy.OutputField(desc="the complete and refined asp program")

    def __init__(self, state):
        super().__init__()
        self.state = state
        self.predictor = dspy.ChainOfThought(self.Signature)

    def forward(self, prompt_1, context, question):
        result = self.predictor(prompt_1=prompt_1, context=context, question=question)
        asp = result.asp
        self.state['convert'] = {
            'context': context,
            'question': question,
            'prompt_1': prompt_1,
            'asp': asp,
        }
        return dspy.Prediction(asp=asp)

class RunClingo(dspy.Module):
    class Signature(dspy.Signature):
        asp_code = dspy.InputField(desc="Initial ASP code to be run and evaluated")
        result = dspy.OutputField(desc="Result from the ASP solver")
        error = dspy.OutputField(desc="Error output from the ASP solver")
    
    def __init__(self, state):
        super().__init__()
        self.state = state
        self.predictor = dspy.Predict(self.Signature)
    
    def forward(self, asp_code):
        result, error = self.run_asp(asp_code)
        self.state['running_asp'] = {
            'result': result,
            "error": error,
        }
        return dspy.Prediction(result=result, error=error)
    
    def run_asp(self, asp_code):
        # Clean the ASP code
        asp_code = asp_code.replace("prolog", "").replace("```", "").replace("\n\n\nQuery:", "").strip()
        # Capture the standard error output
        error_output = StringIO()
        original_stderr = sys.stderr
        sys.stderr = error_output

        try:
            ctl = clingo.Control(["0", "--warn=all"])
            ctl.add("base", [], asp_code)
            
            # Grounding step
            ctl.ground([("base", [])])
            
            results = []
            def on_model(model):
                for atom in model.symbols(shown=True):
                    if atom.name == "query" and len(atom.arguments) > 0:
                        results.append(str(atom.arguments[0]))

            # Solving step
            solve_result = ctl.solve(on_model=on_model)

            if solve_result.satisfiable:
                if results:
                    asp_output = ", ".join(results)
                else:
                    asp_output = "satisfiable, but no query results"
            elif solve_result.unsatisfiable:
                asp_output = "unsatisfiable"
            else:
                asp_output = "unknown"

        except RuntimeError as e:
            asp_output = f"Error: {str(e)}"
        except Exception as e:
            asp_output = f"Unexpected Error: {str(e)}"
        finally:
            sys.stderr = original_stderr

        error_string = error_output.getvalue().strip()
        error_output.close()
        
        return asp_output, error_string

    
class Pipeline(dspy.Module):
    def __init__(self, state):
        super().__init__()
        self.state = state
        self.convert = Convert(state)
        self.clingo = RunClingo(state)  # Initialize RunClingo with state
        
    def forward(self, context, question, prompt_1):
        # Convert the natural language description into ASP facts and query
        convert_result = self.convert.forward(prompt_1=prompt_1, context=context, question=question)
        asp = convert_result.asp
        asp_code = asp
        clingo_result = self.clingo.forward(asp_code=asp_code)
        return dspy.Prediction(asp_code=asp_code, result=clingo_result.result)

def process_examples(examples: List[dspy.Example], pipeline: Pipeline) -> List[Dict[str, Any]]:
    results = []
    for example in examples:
        try:
            context = example.get('context')
            question = example.get('question')
            prompt_1 = example.get('prompt_1')
            q_type=  example.get('q_type')
            prediction = pipeline(context= context, question=question, prompt_1=prompt_1)
           
            result = {
            "context": context,
            "question": question,
            "q_type": q_type,
            "choices":example.get('choices'),
            "predicted_ASP": prediction.asp_code,
            "asp_result": prediction.result,
            "actual_answer": example.get('answer')}
            results.append(result)
     
        except Exception as e:
            print(f"Error processing example: {example.question}")
            print(f"Error: {str(e)}")
    return results

def main(input_file: str, output_file: str):
    df = pd.read_csv(input_file)
    clean_data = df.to_dict(orient='records')

    examples = [
        dspy.Example(
            prompt_1= new_prompt_facts,
            context=r["Story"],
            q_type=r["Q_type"],
            question="".join(r["Question"]),
            choices="".join(r["Candidate_Answers"]),
            answer="".join(r["Answer"])
        ).with_inputs("context", "question", "prompt_1", "choices")
          for r in clean_data  if  not pd.isna(r["Reasoning_type"]) and "Quantifier" in ast.literal_eval(r["Reasoning_type"])
    ]
    print(len(examples))
    state= {}
    pipeline = Pipeline(state)
    results = process_examples(examples, pipeline)

    with open(output_file, "w") as jsonfile:
        json.dump(results, jsonfile, indent=4)

if __name__ == "__main__":
    main("Spar_test.csv", "Spar_human/llama3_quantifier_1_iteration.json")